<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

# 2. Armado del ambiente

## 2.1 Armado del ambiente en CPU

No se requiere ejecuciones previas para la ejecución del programa

##2.2 Armado del ambiente en GPU

Se requiere la instalación del modulo de Cuda para Python

In [1]:
!pip install pycuda

#3. Desarrollo

##3.1 Desarrollo en CPU

In [ ]:
#---------------------PARAMETROS-------------------------------
#@title 3.1.1 Parametros de ejecución {vertical-output: true}

cantidadElementos = 1000000#@param {type: "number"}

import numpy
import time

tiempoTotal = time.time() * 1000

vector = numpy.random.randint(1000, size = cantidadElementos)

#----------------------ALGORITMO----------------------------
tiempoBucle = time.time() *1000

#----------------------FINALIZACION----------------------------
tiempoBucle = (time.time() *1000) - tiempoBucle

tiempoTotal = (time.time() *1000) - tiempoTotal
print("\n")
print("Tiempo del bucle: " , tiempoBucle , "[ms.]")
print("Tiempo del total: " , tiempoTotal , "[ms.]")

Original:
Ordenado:


Tiempo del bucle:  30113.286865234375 [ms.]
Tiempo del total:  30121.781005859375 [ms.]


##3.2 Desarrollo en GPU

In [2]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

orden_polinomio =  2#@param {type: "number"}
orden_taylor = 1#@param {type: "number"}
punto_evalacion =  3#@param {type: "number"}
# --------------------------------------------

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import time
import numpy

tiempoTotal = (time.time() * 1000)

# Vectores en CPU.
coeficientes_cpu = numpy.random.randint(10, size = orden_polinomio + 1 )
coeficientes_cpu = coeficientes_cpu.astype(numpy.float32())

print("Coeficientes: ")
print(coeficientes_cpu)

terminos_taylor_cpu = numpy.zeros(orden_taylor + 1)
terminos_taylor_cpu = terminos_taylor_cpu.astype(numpy.float32())

resultado_cpu = numpy.zeros(orden_taylor + 1)
resultado_cpu = resultado_cpu.astype(numpy.float32())

derivadas_cpu = numpy.arange(orden_polinomio + 1)
derivadas_cpu = derivadas_cpu.astype(numpy.float32())

print("Derivadas")
print(derivadas_cpu)
# Vectores en GPU.
coeficientes_gpu = cuda.mem_alloc(coeficientes_cpu.nbytes)
terminos_taylor_gpu = cuda.mem_alloc(terminos_taylor_cpu.nbytes)
derivadas_gpu = cuda.mem_alloc(derivadas_cpu.nbytes)

# Copio los datos
cuda.memcpy_htod(coeficientes_gpu, coeficientes_cpu)
cuda.memcpy_htod(terminos_taylor_gpu, terminos_taylor_cpu)
cuda.memcpy_htod(derivadas_gpu, derivadas_cpu)

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_taylor( float punto, 
                               int long_pol, 
                               float *C, 
                               float *T,
                               float *D){
  
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  
  float valor = 0;
  int idx_polinomio = 0;
  float aux = 0;
  
  
  if(idx == 0){
    valor += C[idx_polinomio];
    for(idx_polinomio = idx + 1; idx_polinomio < long_pol + 1; idx_polinomio++){
      aux += C[idx_polinomio] * pow((double) punto, (double) idx_polinomio); 
    }
    valor += aux;
  } else{
    valor += C[idx_polinomio];
    for(idx_polinomio = idx + 1; idx_polinomio < long_pol + 1; idx_polinomio++){
      float termino = C[idx_polinomio] * D[idx_polinomio];
      C[idx_polinomio] = termino;
      D[idx_polinomio] = idx_polinomio - 1;
      aux += termino * pow((double) punto, (double) D[idx_polinomio]);
    }
    valor += aux;
  }

  T[idx] = valor;
  return; 
}
""")
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_taylor")

tiempo_gpu = (time.time() * 1000)

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (orden_taylor + 1 + dim_hilo-1) / dim_hilo )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel(numpy.float32(punto_evalacion),
       numpy.float32(orden_polinomio + 1),
       coeficientes_gpu, terminos_taylor_gpu, derivadas_gpu, 
       block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1))

cuda.memcpy_dtoh(resultado_cpu, terminos_taylor_gpu)

tiempo_gpu = (time.time() * 1000) - tiempo_gpu

tiempo_total = (time.time() * 1000) - tiempoTotal
print(resultado_cpu)
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_total, "[ms]" )
print("Tiempo GPU: ", tiempo_gpu, "[ms]" )

Coeficientes: 
[7. 6. 8.]
Derivadas
[0. 1. 2.]


LogicError: ignored

#4. Tabla de pasos

#5. Conclusiones